In [1]:
!pip install -q datasets
!huggingface-cli login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: w

In [7]:
from datasets import load_dataset
import re

# Load the dataset
dataset = load_dataset('timdettmers/openassistant-guanaco')

# shuffle the dataset and slice it.
dataset = dataset['train'].shuffle(seed=42).select(range(1000))
print(dataset['text'][10])

# Define a function to transform the data
def transform_conversation(example):
  conversation_text = example['text']
  segments = conversation_text.split('###')

  reformatted_segments = []

  # Iterate over pairs of segments
  for i in range(1, len(segments) - 1, 2):
    human_text = segments[i].strip().replace('Human:', '').strip()

    # check if there is a corresponding assistant segment before processing
    if i + 1 < len(segments):
      assistant_text = segments[i+1].strip().replace('Assistant:', '').strip()

      # Apply the new template
      reformatted_segments.append(f'<s>[INST] {human_text} [/INST] {assistant_text}</s>')
    else:
      # Handle the case where there is no corresponding assistant segment
      reformatted_segments.append(f'<s>[INST] {human_text} [/INST] </s>')

  return {'text': ''.join(reformatted_segments)}

# Apply the transformation
transformed_dataset = dataset.map(transform_conversation)
print(transformed_dataset['text'][10])



/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


### Human: You will create a table with macronutrients, micronutrients and kcal of the following foods: 100g oatmeal, banana 100g, apple 100g, pasta ragù 100g### Assistant: Here is a table showing the macronutrients, micronutrients, and kcal of 100g of oatmeal, banana, apple, and pasta ragù:

Food	Carbohydrates (g)	Fat (g)	Protein (g)	Fiber (g)	Potassium (mg)	Vitamin C (mg)	kcal
Oatmeal	66.3	6.9	16.9	8.0	-	-	389
Banana	23.0	0.3	1.3	2.6	358	8.7	89
Apple	14.0	0.2	0.3	2.4	107	4.6	52
Pasta Ragù	53.0	12.0	12.0	2.0	-	-	300
Note: The above table is an estimation and may vary slightly depending on the specific### Human: Thanks, that's awesome!### Assistant: No prob, bud!
<s>[INST] You will create a table with macronutrients, micronutrients and kcal of the following foods: 100g oatmeal, banana 100g, apple 100g, pasta ragù 100g [/INST] Here is a table showing the macronutrients, micronutrients, and kcal of 100g of oatmeal, banana, apple, and pasta ragù:

Food	Carbohydrates (g)	Fat (g)	Protein (g

In [9]:
transformed_dataset.push_to_hub("guanaco-llama2-1k")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

RepositoryNotFoundError: ignored